# SVM ALGORITHM FOR LBBB DETECTION

In [23]:
import numpy as np
import matplotlib.pyplot as plt

NormalBeats_LeadI = np.load('array_Normal_beats_leadI.npy')
LBBBBeats_LeadI = np.load('array_LBBB_beats_leadI.npy')
AtrialFibBeats_LeadI = np.load('array_AtrialFib_beats_leadI.npy')


#Data for testing
LBBBBeats_LeadI_test = np.load('array_LBBB_beats_leadI_test.npy') #Dataset
LBBBBeats_LeadI_Bitalino_test = np.load('array_LBBB_beats_leadI_test_Bitalino_versionlarga.npy') #Bitalino


## FEATURE EXTRACTION

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import skew, kurtosis, median_abs_deviation
import pywt

#Defino funcion para la extraccion de caracteristicas
def extract_features(data):
    features = []
    for beat in data:
        """ WAVELET TRANSFORM """ # EXTRACCION DE CARACTERISTICAS (WAVELET DECOMPOSITION): db4, nivel 1 (minimo nivel y el que me da la mejor caracterizacion de un beat de ecg)
        wav_coef = pywt.wavedec(beat, wavelet, level = level); #retorno de coeficiente de aproximacion (approx coeff son los primeros) y coeficientes de detalle (detail coeff son los segundos)
        features.append([np.sqrt(np.mean(np.square(wav_coef[0]))), np.linalg.norm(wav_coef[0],ord=2), np.linalg.norm(wav_coef[0],ord=1), np.max(wav_coef[0]), np.min(wav_coef[0]), np.mean(wav_coef[0]), np.var(wav_coef[0]), np.std(wav_coef[0]), skew(wav_coef[0]), kurtosis(wav_coef[0]), np.ptp(wav_coef[0]), np.cov(wav_coef[0]), np.corrcoef(wav_coef[0]), np.quantile(wav_coef[0], 0.5), np.percentile(wav_coef[0], 50), median_abs_deviation(wav_coef[0]), np.sqrt(np.mean(np.square(wav_coef[1]))), np.linalg.norm(wav_coef[1],ord=2), np.linalg.norm(wav_coef[1],ord=1), np.max(wav_coef[1]), np.min(wav_coef[1]), np.mean(wav_coef[1]), np.var(wav_coef[1]), np.std(wav_coef[1]), skew(wav_coef[1]), kurtosis(wav_coef[1]), np.ptp(wav_coef[1]), np.cov(wav_coef[1]), np.corrcoef(wav_coef[1]), np.quantile(wav_coef[1], 0.5), np.percentile(wav_coef[1], 50), median_abs_deviation(wav_coef[1])])

    return np.array(features)

wavelet = 'db4'
level = 1

# Extraccion de caracteristicas de cada tipo de beat
NormalBeats_LeadI_features = extract_features(NormalBeats_LeadI)
LBBBBeats_LeadI_features = extract_features(LBBBBeats_LeadI)
AtrialFibBeats_LeadI_features = extract_features(AtrialFibBeats_LeadI)

## SVM ALGORITHM 

In [25]:
""" SVM ALGORITHM """

''' LABELS '''
NormalBeats_LeadI_labels = np.zeros(len(NormalBeats_LeadI_features))
LBBBBeats_LeadI_labels = np.ones(len(LBBBBeats_LeadI_features))
AtrialFibBeats_LeadI_labels = np.full(len(AtrialFibBeats_LeadI_features), 2)

# Concateno caracteristicas y labels
features = np.concatenate((NormalBeats_LeadI_features, LBBBBeats_LeadI_features, AtrialFibBeats_LeadI_features), axis=0)
labels = np.concatenate((NormalBeats_LeadI_labels, LBBBBeats_LeadI_labels, AtrialFibBeats_LeadI_labels), axis=0)

''' SPLITING THE DATA '''
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.15, random_state = 42)

''' Training '''
svm = OneVsOneClassifier(SVC(C = 1000, kernel='linear', decision_function_shape='ovo'))

# Entrenando el clasificador
svm.fit(X_train, y_train)

# Prediccion con el set de testeo
y_pred = svm.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision_per_label = precision_score(y_test, y_pred, average=None)
precision_av = precision_score(y_test, y_pred, average="weighted")
print("Precision por clase:", precision_per_label)
print("Precision promedio:", precision_av)

recall_per_label = recall_score(y_test, y_pred, average=None) #Sensitivity 
recall_av = recall_score(y_test, y_pred, average="weighted") #Sensitivity 
print("Recall por clase:", recall_per_label)
print("Recall promedio:", recall_av)

f1_per_label = f1_score(y_test, y_pred, average=None)
f1_av = f1_score(y_test, y_pred, average="weighted")
print("F1 Score por clase:", f1_per_label)
print("F1 Score promedio:", f1_av)



Accuracy: 0.8254364089775561
Precision por clase: [0.752      0.98387097 0.75657895]
Precision promedio: 0.8259232902463705
Recall por clase: [0.71212121 0.976      0.79861111]
Recall promedio: 0.8254364089775561
F1 Score por clase: [0.73151751 0.97991968 0.77702703]
F1 Score promedio: 0.8252921770955012


## TESTING THE SVM ALGORITHM WITH LBBB DATASET

In [26]:
for test_beat in LBBBBeats_LeadI_test:
    # Extraigo caracteristicas para cada beat
    test_features = extract_features([test_beat])
    
    # Prediccion
    prediction = svm.predict(test_features)
    
    # Determino etiquetas en palabras y ya no numericas
    if prediction == 0:
        label = "Normal"
    elif prediction == 1:
        label = "LBBB"
    else:
        label = "AtrialFib"
    
    #print("Predicted label:", label);

## TESTING THE SVM ALGORITHM WITH DATA ACQUIRED WITH BITALINO KIT

In [27]:
for test_beat_Bit in LBBBBeats_LeadI_Bitalino_test:
    test_features_Bit = extract_features([test_beat_Bit])
    
    prediction = svm.predict(test_features_Bit)
    
    if prediction == 0:
        label = "Normal"
    elif prediction == 1:
        label = "LBBB"
    else:
        label = "AtrialFib"
    
    print("Predicted label:", label);

Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
Predicted label: LBBB
